<div style="font-size:450%;font-weight:600;text-align:center;color:#0277bd;height:40px;margin-top:20px;">
    1<span style="font-size:60%;color:red;">.</span> Quick Look
</div>

# Libraries

In [1]:
# standard
from pathlib import Path

# data
import numpy as np
import pandas as pd

# Config

Load the class `ScriptLoader` to easily load other scripts into notebook's namespace (now let's assume `scripts = ScriptLoader()`).

After loading a script (`scripts.load` method), you can have a look at the different object names loaded using `scripts.listing`.

You can safely use the `load` method as much as you want, it will not load the file again if it has not changed since last loading.

In [2]:
exec(Path('scripts/loader.py').read_bytes())    # this will load the ScriptLoader class into global namespace
scripts = ScriptLoader(prefix='scripts')  # each script path, if not absolute, will be prefixed with `prefix` value

In [3]:
scripts.load('data')  # you can omit the ".py" extension

script "data.py" read and loaded into namespace


# Data load

In [4]:
# `read_data` is in `data.py`
data = read_data(mmap=True)

In [5]:
data.head()

Source System             Product  Underwriting Year      Effective Date  \
0  Simply Business  Maltings Tradesman               2014 2014-09-19 01:00:00   
1  Simply Business  Maltings Tradesman               2014 2014-10-08 01:00:00   
2  Simply Business  Maltings Tradesman               2014 2014-09-18 01:00:00   
3  Simply Business  Maltings Tradesman               2014 2014-10-03 01:00:00   
4  Simply Business  Maltings Tradesman               2014 2014-09-20 01:00:00   

          Expiry Date Transaction Type  Public Liability Limit  \
0 2015-09-18 01:00:00     New Business                 1000000   
1 2015-10-07 01:00:00     New Business                 5000000   
2 2015-09-17 01:00:00     New Business                 5000000   
3 2015-10-02 01:00:00     New Business                 2000000   
4 2015-09-19 01:00:00     New Business                 1000000   

   Employers Liability Limit  Tools Sum Insured  Professional Indemnity Limit  \
0                          0                0.0                         50000   
1                          0                0.0                             0   
2                   10000000             4000.0                             0   
3                          0             2000.0                             0   
4                          0                0.0                       1000000   

            ...                       Location  \
0           ...                       BRIGHTON   
1           ...                        BRISTOL   
2           ...                    NORTHAMPTON   
3           ...                      CAMBRIDGE   
4           ...            NEWCASTLE UPON TYNE   

   Public_Liability_Limit_5000000.2  Public_Liability_Limit_5000000.3  \
0                                 0                                 0   
1                                 1                                 0   
2                                 1                                 0   
3                                 0                                 0   
4                                 0                                 0   

  Professional_Indemnity_Limit_g Risk_Postcode2  TotalEmployees  Claim Count  \
0                              1         BN21 3               1            0   
1                              0         BS15 4               1            0   
2                              0         NN10 8               8            0   
3                              0          CB8 7              10            0   
4                              1          NE5 1               1            0   

   Claim Incurred Capped Incurred (£50k) Capped Incurred (£100k)  
0             0.0                    0.0                     0.0  
1             0.0                    0.0                     0.0  
2             0.0                    0.0                     0.0  
3             0.0                    0.0                     0.0  
4             0.0                    0.0                     0.0  

[5 rows x 67 columns]

# Data inspection: types/formats, features set, target

## shape

In [6]:
m, n = data.shape   # m = nb of data sample, n = nb of features
m, n

(232962, 67)

## types

In [7]:
dtypes = defaultdict(list)
for _name, _type in data.dtypes.iteritems():
    dtypes[_type].append(_name)
for _t in list(dtypes):
    dtypes[str(_t)] = dtypes[_t]
print(f'there are {len(dtypes)} different types')
for _type, _list in dtypes.items():
    if isinstance(_type, str):
        continue
    _n = len(_list)
    print(f'   - {_n:2d} ({_n/n*100.:4.1f}%) features of type "{_type}"')

there are 8 different types
   - 10 (14.9%) features of type "object"
   - 44 (65.7%) features of type "int64"
   -  3 ( 4.5%) features of type "datetime64[ns]"
   - 10 (14.9%) features of type "float64"


In [8]:
# text features
for _col in dtypes['object']:
    _c = data.loc[:, _col].value_counts()
    if _c.size > 20:
        print(f'{_col:.<35s} {_c.size}')
    else:
        print(f'     {_col:.<30s} {_c.size}')

     Source System................. 2
     Product....................... 6
     Transaction Type.............. 3
Trade 1............................ 880
Trade 2............................ 759
     Match Type.................... 8
     Trade 1 Category.............. 16
     Trade 2 Category.............. 17
Location........................... 121
Risk_Postcode2..................... 20857


<span style="color: red;font-weight: 600;font-size: 150%;">==> Thus:</span>

`Location` is clearly a categorical, and `Risk_Postcode2` as well according to the data dictionary.

`Trade 1` and `Trade 2` might benefit from text preprocessing, even maybe a subset of NLP techniques. But `Trade 1/2 Category` features might be a good enough proxy, thus allowing to drop these 2 more detailed features all together... (to be checked)

## target

In [9]:
scripts.load('features')

script "features.py" read and loaded into namespace


In [10]:
target.name

'Claim Count'

In [11]:
# `target` is in `features.py`
claim_counts = data.loc[:, target.name].value_counts()
claim_counts

0    229689
1      3098
2       150
3        22
4         3
Name: Claim Count, dtype: int64

<span style="color: red;font-weight: 600;font-size: 150%;">==> Thus:</span>

This is an **imbalanced problem**. Ratios are:

In [12]:
print(f'no claims represent {claim_counts[0]/m*100:.4g}% of the data')
print(f'claims (regardless the number) represent {claim_counts[1:].sum()/m*100:.4g}% of the data')
for _i in range(1, claim_counts.size):
    print(f'   - {_i:d} claim(s) represent {claim_counts[_i]/m*100:.4g}% of the data')

no claims represent 98.6% of the data
claims (regardless the number) represent 1.405% of the data
   - 1 claim(s) represent 1.33% of the data
   - 2 claim(s) represent 0.06439% of the data
   - 3 claim(s) represent 0.009444% of the data
   - 4 claim(s) represent 0.001288% of the data
